What are the three longest trips on rainy days?

WITH rainy as 
(
SELECT 
    DATE(date) rain_date
From weather
WHERE events = 'Rain'
GROUP BY 1
) 

SELECT
    t.trip_id,
    t.duration,
    DATE(t.start_date)
FROM trips t
JOIN rainy r
on DATE(t.start_date) = r.rain_date
ORDER BY duration DESC
LIMIT 3

Which station is full most often?

SELECT
    status.station_id,
    stations.name,
    COUNT(CASE WHEN docks_available=0 then 1 END) empty_count
FROM status
JOIN stations
on stations.station_id = status.station_id
GROUP BY 1,2
ORDER BY empty_count DESC

Return a list of stations with a count of number of trips starting at that station but ordered by dock count.

SELECT
    start_station,
    dockcount,
    COUNT(*)
FROM trips
JOIN stations
ON stations.name=trips.start_station
GROUP BY 1, 2
ORDER BY 2 DESC

(Challenge) What's the length of the longest trip for each day it rains anywhere?

WITH rainy as 
(
SELECT 
    DATE(date) weather_date
FROM weather
WHERE events = 'Rain'
GROUP BY 1
),
rain_trips as (
SELECT
    trip_id,
    duration,
    DATE(trips.start_date) rain_trips_date
FROM trips
JOIN rainy
on rainy.weather_date = DATE(trips.start_date)
ORDER BY duration DESC
)
SELECT 
    rain_trips_date,
    max(duration) max_duration
FROM rain_trips
GROUP BY 1
ORDER BY max_duration DESC

By default, SQL will perform an inner join. This means that the rows returned will have a successful join between the tables. 

If there is no match between the given columns, the rows will not be returned.

 let's join the station and trip tables to return the latitude and longitude for the start station for every trip, along with the trip id.
 
 SELECT
    trips.trip_id,
    trips.start_station,
    stations.lat,
    stations.long
FROM
    trips 
JOIN
    stations
ON
    trips.start_station = stations.name;

Let's rewrite the query above to demonstrate table aliases, a commonly used SQL feature. This query is exactly the same:

SELECT
    t.trip_id,
    t.start_station,
    s.lat,
    s.long
FROM
    trips t
JOIN
    stations s
ON
    t.start_station = s.name;

by default, SQL will perform an inner join

rows are only returned when there is a match on both sides, it doesn't matter which table is on the left and which table is on the right.

there are also three types of outer joins: left outer joins, right outer joins, and full outer joins.

In a left outer join every row from the left table will be included in your output, even if there was no matching row on the right table.
Rows without a match will be filled with NULL for the columns from the right table.
Left outer joins are often called just "left joins", and you can perform a left join using LEFT OUTER JOIN or just LEFT JOIN.

A right outer join, or "right join" is exactly the same as a left join, except that all the rows from the right table are returned, even if there is no match. 
Because the only difference is table order, you could reverse the left and right tables and use a left join instead to accomplish the same thing. 

A full outer join, also known as a "full join" or just an "outer join" returns all matching records from both the left and right tables. This can potentially return very large data sets,

CTEs (common table expressions)

the result of every SQL query is itself a table

That means you can use joins not just to join tables on existing tables, but also to join them on the results of other queries

One way to do that is to use common table expressions, or CTE's

Recall before when we generated the average latitude and longitude of every city. What if we wanted to also include a count of the number of trips that started in each of those cities?

if we tried to do this through a single query we might try something like this:

SELECT
    s.city,
    AVG(s.lat) lat,
    AVG(s.long) long,
    COUNT(*)
FROM
    stations s
JOIN
    trips t
ON
    t.start_station = s.name 
GROUP BY 1;

this query is actually incorrect. 

When working with JOINs, the join happens before any aggregate functions.

This does not take the average lat/long for every station.  It takes the average for every trip.
We don't want the average of the joined data, instead we want to get the average and then join the data.

CTE's start with the form WITH __expression__ as (...)

This will create another, intermediate table for you to work with and join on. 

-- Set up the CTE to create a "locations" table.
WITH
    locations
AS (
    -- A simple query to get the averages of lat and long on a city level.
    SELECT
        city,
        AVG(lat) lat,
        AVG(long) long
    FROM
        stations
    GROUP BY 1
)

-- Joining the locations table we created with the trips table to count trips.
SELECT
    l.city,
    l.lat,
    l.long,
    COUNT(*)
FROM
    locations l

-- We need an intermediate join to go from locations to stations 
-- because the trips table does not have a "city" column.
JOIN
    stations s
ON
    l.city = s.city
JOIN
    trips t
ON
    t.start_station = s.name
GROUP BY 1,2,3;

Firstly, the CTE, locations, groups stations by city name to find the average of the coordinates. 

 You are then creating a new temporary locations table with an entry for lat and long for each city. 
 
 Then we want to join the location table with the trips table.
 
 But trips does not have a city column to use as a key to join with location.
 
 instead, location has a city
 station has a city
 stations has a name
 trip has a start station name
 
 so connect locations to station by city name
 then connect station to trips by station name
 
 this creates a connection from location to trip
 
 location has the average lat/long of each station
 and that can link to each trip
 without taking the average lat/long for each trip

CASE statements allow you to set up conditions and then take action in a column based on them

It is also common to combine CASE statements with COUNT to do conditional counts.

he most common form for case statements is 
CASE WHEN __condition__ THEN __value__ ELSE __value__ END

SELECT
    (CASE WHEN dockcount > 20 THEN 'large' ELSE 'small' END) station_size,
    COUNT(*) as station_count
FROM 
    stations
GROUP BY 1;

This CASE statement looks at the stations table and labels each row either 'large' or 'small' depending on the value of dockcount for that row, and then counts how many rows there are for each case.

The group by statement makes it so we are counting based on the station size.